In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from model import RandomForest
np.random.seed(42)
torch.random.manual_seed(42)

# Titanic data

In [2]:
path_csv = "data/titanic_modified_dataset.csv"
titanic_data_df = pd.read_csv(path_csv, index_col="PassengerId")
titanic_data_df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Survived
PassengerId,,,,,,,,,
1,3,0,22.0,1,0,7.2500,0,0,0
2,1,1,38.0,1,0,71.2833,1,1,1
3,3,1,26.0,0,0,7.9250,0,2,1
4,1,1,35.0,1,0,53.1000,0,1,1
5,3,0,35.0,0,0,8.0500,0,0,0
...,...,...,...,...,...,...,...,...,...
887,2,0,27.0,0,0,13.0000,0,5,0
888,1,1,19.0,0,0,30.0000,0,2,1
889,3,1,28.0,1,2,23.4500,0,2,0


In [3]:
# convert to numpy array
titanic_data_arr = titanic_data_df.to_numpy().astype(np.float32)

# shuffle data
idx = np.arange(titanic_data_arr.shape[0])
np.random.shuffle(idx)
titanic_data_arr = titanic_data_arr[idx]

In [4]:
# devide features to X, label to y 
X, y = titanic_data_arr[:, :-1], titanic_data_arr[:, -1]
print(X.shape)
print(y.shape)

(891, 8)
(891,)


In [5]:
TRAIN_SIZE = 0.7
VAL_SIZE = 0.2
#TEST_SIZE = 0.1
TRAIN_SAMPLES = int(TRAIN_SIZE * titanic_data_arr.shape[0])
VAL_SAMPLES = TRAIN_SAMPLES + int(VAL_SIZE * titanic_data_arr.shape[0])

X_train, y_train = X[:TRAIN_SAMPLES], y[:TRAIN_SAMPLES]
x_val, y_val = X[TRAIN_SAMPLES:VAL_SAMPLES], y[TRAIN_SAMPLES:VAL_SAMPLES]
X_test, y_test = X[VAL_SAMPLES:], y[VAL_SAMPLES:]

print(X_train.shape, x_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(623, 8) (178, 8) (90, 8)
(623,) (178,) (90,)


## Impurity function - Entropy

In [6]:
RandomForestModel = RandomForest(n_trees=25, min_sample=2, max_depth=10, impurity_function="entropy")
RandomForestModel.fit(X_train, y_train)
y_pred = RandomForestModel.predict(X_test)
print(y_pred)
print("Accuracy: ", accuracy_score(y_test, y_pred))

tensor([0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1.,
        0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1.])
Accuracy:  0.6111111111111112


In [7]:
RandomForestModel.forest[0].print_tree(RandomForestModel.forest[0].root, list_feature=titanic_data_df.columns[:-1])

                                                                ┌0.0
                                                      ┌Age, 19.0┤
                                                      |         |         ┌1.0
                                                      |         └Age, 20.0┤
                                                      |                   |                               ┌1.0
                                                      |                   |                     ┌Age, 27.0┤
                                                      |                   |                     |         └0.0
                                                      |                   |          ┌Fare, 6.97┤
                                                      |                   |          |          └0.0
                                                      |                   └Fare, 7.74┤
                                                      |                              |        

## Impurity function - Gini

In [8]:
RandomForestModel = RandomForest(n_trees=25, min_sample=2, max_depth=10, impurity_function="gini")
RandomForestModel.fit(X_train, y_train)
y_pred = RandomForestModel.predict(X_test)
print(y_pred)
print("Accuracy: ", accuracy_score(y_test, y_pred))

tensor([0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1.,
        0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0.,
        0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1.])
Accuracy:  0.7


In [9]:
RandomForestModel.forest[19].print_tree(RandomForestModel.forest[0].root, list_feature=titanic_data_df.columns[:-1])

                                         ┌1.0
                              ┌SibSp, 1.0┤
                              |          └0.0
                     ┌Age, 9.0┤
                     |        |                               ┌0.0
                     |        |                    ┌Fare, 6.95┤
                     |        |                    |          └1.0
                     |        |         ┌Fare, 6.97┤
                     |        |         |          |         ┌0.0
                     |        |         |          └Age, 31.0┤
                     |        |         |                    |                    ┌0.0
                     |        |         |                    |         ┌Fare, 7.75┤
                     |        |         |                    |         |          |          ┌1.0
                     |        |         |                    |         |          └Fare, 7.85┤
                     |        |         |                    |         |                 